# Heat Consumption Forecasting Benchmarker
## End-to-End Model Comparison: NHITS vs TimesNet

This notebook provides a comprehensive benchmarking workflow for comparing NHITS and TimesNet models for heat consumption forecasting. It includes:
- Model training and evaluation
- Performance metrics comparison (MAE, RMSE, MAPE, PICP, MIW, CRPS)
- Advanced visualizations
- Reproducible results

**Dataset**: Nordbyen heat consumption data with weather and temporal features  
**Models**: NHITS (Darts) and TimesNet (NeuralForecast)  
**Evaluation**: 50-day walk-forward validation with probabilistic forecasting

## 1. Import Required Libraries
Import all necessary modules for benchmarking, visualization, and data processing.

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from IPython.display import display, Image

# Import benchmarker module
from benchmarker import Benchmarker

# Set visualization style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries imported successfully")
print(f"Current working directory: {os.getcwd()}")

/home/hpc/iwi5/iwi5389h/.conda/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-20 20:41:48,467	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-12-20 20:41:49,007	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


✓ All libraries imported successfully
Current working directory: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis


## 2. Configure Benchmark Parameters
Set up paths and parameters for the benchmarking process.

In [2]:
# Data and model paths
DATA_PATH = "nordbyen_features_engineered.csv"
RESULTS_DIR = "results"
MODELS_DIR = "models"

# Models to benchmark
MODELS_TO_RUN = ["NHITS", "TIMESNET"]

# Time periods for training/validation/testing
TRAIN_END = "2018-12-31 23:00:00+00:00"
VAL_END = "2019-12-31 23:00:00+00:00"
TEST_START = "2020-01-01 00:00:00+00:00"

# Verify data file exists
if os.path.exists(DATA_PATH):
    print(f"✓ Data file found: {DATA_PATH}")
    df_info = pd.read_csv(DATA_PATH, nrows=5)
    print(f"  Columns: {list(df_info.columns[:5])}... (total: {len(df_info.columns)})")
else:
    print(f"✗ Data file not found: {DATA_PATH}")
    
# Create results directory if needed
os.makedirs(RESULTS_DIR, exist_ok=True)
print(f"✓ Results directory ready: {RESULTS_DIR}")

✓ Data file found: nordbyen_features_engineered.csv
  Columns: ['timestamp', 'heat_consumption', 'temp', 'dew_point', 'humidity']... (total: 27)
✓ Results directory ready: results


## 3. Run Benchmarker for Both Models

**Two execution options:**
1. **SLURM (Recommended)**: Submit job to HPC cluster - run cells 7A and 7B
2. **Local**: Run directly in notebook - run cell 7C (takes 10-15 minutes)

### Option A: Submit SLURM Job (Recommended)

### Option C: Run Locally (Alternative)

⚠️ **Only use this if you don't want to use SLURM.** This will run in the notebook kernel and may take 10-15 minutes.

### Option B: Monitor SLURM Job Status

Run this cell to check if the benchmarker job is complete. Once complete, proceed to cell 8.

In [ ]:
# Submit benchmarker to SLURM
import subprocess
import time

print("Submitting benchmarker job to SLURM...")
result = subprocess.run(
    ["sbatch", "benchmark_job.slurm"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    # Extract job ID from output: "Submitted batch job 123456"
    job_id = result.stdout.strip().split()[-1]
    print(f"✓ Job submitted successfully!")
    print(f"  Job ID: {job_id}")
    print(f"  Log files: benchmark_{job_id}.log / benchmark_{job_id}.err")
    print(f"\n💡 Next: Run cell 7B to monitor job status")
    
    # Save job ID for monitoring
    with open("current_job_id.txt", "w") as f:
        f.write(job_id)
else:
    print(f"✗ Error submitting job:\n{result.stderr}")

In [7]:
# Check SLURM job status
import subprocess
import os
from datetime import datetime

# Get job ID
if os.path.exists("current_job_id.txt"):
    with open("current_job_id.txt", "r") as f:
        job_id = f.read().strip()
else:
    print("❌ No job ID found. Run cell 7A first to submit the job.")
    job_id = None

if job_id:
    print(f"Checking status of Job {job_id}...")
    print("="*70)
    
    # Check with squeue
    result = subprocess.run(
        ["squeue", "-j", job_id, "--format=%.18i %.9P %.20j %.8u %.8T %.10M %.9l %.6D %R"],
        capture_output=True,
        text=True
    )
    
    if "Invalid job id" in result.stderr or not result.stdout.strip().split('\n')[1:]:
        print(f"⏹️  Job {job_id} is no longer in queue (completed or failed)")
        
        # Check if results exist
        results_file = os.path.join(RESULTS_DIR, "benchmark_results.csv")
        if os.path.exists(results_file):
            print(f"\n✅ Job completed successfully!")
            print(f"   Results file found: {results_file}")
            print(f"\n💡 Next: Continue to cell 8 to view results")
        else:
            print(f"\n❌ Job completed but results not found!")
            print(f"   Check log files: benchmark_{job_id}.log / benchmark_{job_id}.err")
    else:
        print(result.stdout)
        print(f"\n⏳ Job is still running. Re-run this cell to check again.")
        print(f"   Estimated time: 10-15 minutes total")
        
    # Show recent log output
    log_file = f"benchmark_{job_id}.log"
    if os.path.exists(log_file):
        print(f"\n📄 Recent log output (last 15 lines):")
        print("-"*70)
        result = subprocess.run(["tail", "-n", "15", log_file], capture_output=True, text=True)
        print(result.stdout)

Checking status of Job 1471438...
             JOBID PARTITION                 NAME     USER    STATE       TIME TIME_LIMI  NODES NODELIST(REASON)
           1471438   rtx3080 benchmark_nhits_time iwi5389h  RUNNING       3:03   3:00:00      1 tg081


⏳ Job is still running. Re-run this cell to check again.
   Estimated time: 10-15 minutes total

📄 Recent log output (last 15 lines):
----------------------------------------------------------------------

Validation DataLoader 0:  95%|█████████▌| 255/268 [00:01<00:00, 185.84it/s]

Validation DataLoader 0:  96%|█████████▌| 256/268 [00:01<00:00, 185.71it/s]

Validation DataLoader 0:  96%|█████████▌| 257/268 [00:01<00:00, 185.58it/s]

Validation DataLoader 0:  96%|█████████▋| 258/268 [00:01<00:00, 185.46it/s]

Validation DataLoader 0:  97%|█████████▋| 259/268 [00:01<00:00, 185.34it/s]

Validation DataLoader 0:  97%|█████████▋| 260/268 [00:01<00:00, 185.22it/s]

Validation DataLoader 0:  97%|█████████▋| 261/268 [00:01<00:00, 185.10it/s]

Vali

### Option C: Run Locally (Alternative)

⚠️ **Only use this if you don't want to use SLURM.** This will run in the notebook kernel and may take 10-15 minutes.

In [6]:
# # LOCAL EXECUTION - Run benchmarker directly in notebook
# print("="*70)
# print("Starting Benchmark Pipeline (LOCAL EXECUTION)")
# print("="*70)
# print("\n⚠️  This will take 10-15 minutes and may timeout")
# print("    - NHITS: ~2-3 minutes (15 epochs)")
# print("    - TimesNet: ~3-5 minutes (50 epochs)")
# print("\n💡 RECOMMENDED: Use SLURM instead (cells 7A & 7B)")
# print("="*70 + "\n")

# benchmarker = Benchmarker(DATA_PATH, MODELS_TO_RUN)
# benchmarker.run()

# print("\n" + "="*70)
# print("Benchmark Complete!")
# print("="*70)

## 4. Display Benchmark Results
Load and display the comprehensive metrics comparison.

In [ ]:
# Load and display results
results_file = os.path.join(RESULTS_DIR, "benchmark_results.csv")

if not os.path.exists(results_file):
    print("❌ ERROR: Benchmark results not found!")
    print(f"   Expected file: {results_file}")
    print("\n💡 Solution:")
    print("   - If using SLURM: Check cell 7B to see if job is still running")
    print("   - If using local: Run cell 7C first to execute the benchmarker")
else:
    results_df = pd.read_csv(results_file)
    
    print("\n📊 BENCHMARK RESULTS SUMMARY")
    print("="*70)
    display(results_df)
    
    # Highlight best performer for each metric
    print("\n🏆 Best Performer by Metric:")
    for metric in ['MAE', 'RMSE', 'MAPE', 'MIW', 'CRPS']:
        if metric in results_df.columns:
            best_model = results_df.loc[results_df[metric].idxmin(), 'Model']
            best_value = results_df[metric].min()
            print(f"  {metric:8s}: {best_model:10s} ({best_value:.4f})")
    
    # For PICP, higher is better (should be close to 80% for 80% prediction interval)
    if 'PICP' in results_df.columns:
        target_picp = 80.0
        results_df['PICP_diff'] = (results_df['PICP'] - target_picp).abs()
        best_model = results_df.loc[results_df['PICP_diff'].idxmin(), 'Model']
        best_value = results_df.loc[results_df['PICP_diff'].idxmin(), 'PICP']
        print(f"  PICP    : {best_model:10s} ({best_value:.2f}% - closest to 80%)")

## 5. Generate Visualizations
Create comprehensive visualizations comparing both models.

In [ ]:
# Run visualization script
print("Generating visualizations...")
import subprocess
result = subprocess.run(
    ["python3", "visualize_benchmark.py"],
    capture_output=True,
    text=True
)
print(result.stdout)
if result.returncode == 0:
    print("✓ Visualizations generated successfully!")
else:
    print(f"✗ Error generating visualizations:\n{result.stderr}")

### 5.1 Metrics Comparison Bar Plots
Compare all 6 metrics (MAE, RMSE, MAPE, PICP, MIW, CRPS) across both models.

In [ ]:
metrics_plot = os.path.join(RESULTS_DIR, "benchmark_metrics_barplots.png")
if os.path.exists(metrics_plot):
    display(Image(filename=metrics_plot))
else:
    print(f"Plot not found: {metrics_plot}")

### 5.2 Box Plots - Prediction Distributions
Compare the distribution of actual values and predictions (p10, p50, p90) for both models.

In [ ]:
boxplot_img = os.path.join(RESULTS_DIR, "benchmark_boxplots.png")
if os.path.exists(boxplot_img):
    display(Image(filename=boxplot_img))
else:
    print(f"Plot not found: {boxplot_img}")

### 5.3 Side-by-Side Time Series Comparison
Visual comparison of forecast quality over the first 7 days of testing.

In [ ]:
sidebyside_img = os.path.join(RESULTS_DIR, "benchmark_comparison_sidebyside.png")
if os.path.exists(sidebyside_img):
    display(Image(filename=sidebyside_img))
else:
    print(f"Plot not found: {sidebyside_img}")

## 6. Individual Model Visualizations

### 6.1 NHITS Model Analysis

In [ ]:
print("NHITS - Time Series Forecast")
nhits_ts = os.path.join(RESULTS_DIR, "nhits_timeseries.png")
if os.path.exists(nhits_ts):
    display(Image(filename=nhits_ts))

print("\nNHITS - Error Histogram")
nhits_err = os.path.join(RESULTS_DIR, "nhits_error_hist.png")
if os.path.exists(nhits_err):
    display(Image(filename=nhits_err))

print("\nNHITS - Scatter Plot")
nhits_scatter = os.path.join(RESULTS_DIR, "nhits_scatter.png")
if os.path.exists(nhits_scatter):
    display(Image(filename=nhits_scatter))

### 6.2 TimesNet Model Analysis

In [ ]:
print("TimesNet - Time Series Forecast")
timesnet_ts = os.path.join(RESULTS_DIR, "timesnet_timeseries.png")
if os.path.exists(timesnet_ts):
    display(Image(filename=timesnet_ts))

print("\nTimesNet - Error Histogram")
timesnet_err = os.path.join(RESULTS_DIR, "timesnet_error_hist.png")
if os.path.exists(timesnet_err):
    display(Image(filename=timesnet_err))

print("\nTimesNet - Scatter Plot")
timesnet_scatter = os.path.join(RESULTS_DIR, "timesnet_scatter.png")
if os.path.exists(timesnet_scatter):
    display(Image(filename=timesnet_scatter))

## 7. Results Interpretation

### Key Findings:

In [ ]:
# Calculate percentage differences
if not os.path.exists(os.path.join(RESULTS_DIR, "benchmark_results.csv")):
    print("❌ Cannot analyze results - benchmark hasn't been run yet!")
    print("   Please run cell 7 first.")
else:
    results_df = pd.read_csv(os.path.join(RESULTS_DIR, "benchmark_results.csv"))
    results_df_sorted = results_df.set_index('Model')
    
    if 'NHITS' not in results_df_sorted.index or 'TIMESNET' not in results_df_sorted.index:
        print("❌ Both models haven't been benchmarked yet!")
        print("   Current models in results:", list(results_df_sorted.index))
    else:
        nhits_metrics = results_df_sorted.loc['NHITS']
        timesnet_metrics = results_df_sorted.loc['TIMESNET']
        
        print("📈 Performance Analysis:")
        print("="*70)
        
        for metric in ['MAE', 'RMSE', 'MAPE', 'MIW', 'CRPS']:
            if metric in nhits_metrics and metric in timesnet_metrics:
                diff = ((timesnet_metrics[metric] - nhits_metrics[metric]) / nhits_metrics[metric]) * 100
                better = "NHITS" if diff > 0 else "TIMESNET"
                print(f"\n{metric}:")
                print(f"  NHITS    : {nhits_metrics[metric]:.4f}")
                print(f"  TimesNet : {timesnet_metrics[metric]:.4f}")
                print(f"  Difference: {abs(diff):.2f}% (better: {better})")
        
        # PICP analysis
        if 'PICP' in nhits_metrics and 'PICP' in timesnet_metrics:
            print(f"\nPICP (Coverage - target 80%):")
            print(f"  NHITS    : {nhits_metrics['PICP']:.2f}%")
            print(f"  TimesNet : {timesnet_metrics['PICP']:.2f}%")
            nhits_diff = abs(nhits_metrics['PICP'] - 80)
            timesnet_diff = abs(timesnet_metrics['PICP'] - 80)
            better = "NHITS" if nhits_diff < timesnet_diff else "TIMESNET"
            print(f"  Better calibrated: {better}")
        
        print("\n" + "="*70)

## 8. Summary and Conclusions

This comprehensive benchmark compared NHITS and TimesNet for heat consumption forecasting:

**Metrics Evaluated:**
- **MAE/RMSE/MAPE**: Point forecast accuracy
- **CRPS**: Probabilistic forecast quality (lower is better)
- **PICP**: Prediction interval coverage (should be ~80% for 80% intervals)
- **MIW**: Mean interval width (narrower intervals are better if coverage is maintained)

**Model Characteristics:**
- **NHITS (Darts)**: Hierarchical temporal neural network with quantile regression
- **TimesNet (NeuralForecast)**: Temporal 2D-variation modeling with multi-horizon quantile loss

**Next Steps:**
- Review individual model plots for detailed error analysis
- Consider hyperparameter optimization if results need improvement
- Examine specific time periods where models differ significantly
- Check feature importance for model interpretability